# Proof of concept of Emissions Prediction Machine Learning Model
The first version will be implemented with scikit-learn



In [32]:
import os
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression


# Read the CSV file
data = pd.read_csv('./data/emissions.csv')

# Drop unnecessary columns
data = data.drop(columns=['Sector', 'Parent sector'])

print(data)


# Melt the dataframe to have a long format with 'year' and 'emissions' columns
#TODO: this seems like magic, find out how it works. It just modifies the dataframe getting all columns that represent years and putting them in a single column,
#      same for emissions
data = data.melt(id_vars=['Country'], var_name='year', value_name='emissions')

print(data)
#
# # Convert the year column to integer
# data['year'] = data['year'].astype(int)
# # Plot the data with X axis as year and Y axis as emissions, and each row that represents a country is a different color, also add some identifying information
# #TODO: investigate how can I add a different color for each country and also some identifying information
# data.plot(x='year', y='emissions', kind='scatter', colormap='viridis', title='Emissions by country and year')



     Country      1850      1851      1852      1853      1854      1855  \
0        ABW  0.041900  0.044100  0.046500  0.049000  0.051700  0.054500   
1        AFG  0.080300  0.080300  0.080300  0.080300  0.080400  0.080400   
2        AGO -0.611000 -0.484000 -0.420000 -0.447000 -0.410000 -0.495000   
3        AIA  0.000164  0.000173  0.000182  0.000192  0.000203  0.000214   
4        ALB  2.110000  2.110000  2.100000  2.100000  2.100000  2.100000   
...      ...       ...       ...       ...       ...       ...       ...   
4843     WSM  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4844     YEM  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4845     ZAF -0.181000 -0.156000 -0.144000 -0.149000 -0.142000 -0.158000   
4846     ZMB  0.709000  0.764000  0.791000  0.780000  0.796000  0.759000   
4847     ZWE  0.124000  0.133000  0.138000  0.136000  0.139000  0.132000   

          1856      1857     1858  ...     2005     2006     2007     2008  \
0     0.0

In [ ]:
# Split the data into input features and output, and then into training and test sets.

from sklearn.model_selection import train_test_split

# Split data into input features and output
X = data[['Country', 'year']]
y = data['emissions']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode the categorical data:
used to perform one-hot encoding on the 'Country' variable in a pandas DataFrame, which is a common technique for converting categorical data into a numerical format that can be used in machine learning. One-hot encoding works by creating new binary columns for each unique category in the original column. The OneHotEncoder class from the sklearn.preprocessing module is used to perform the one-hot encoding.

The fit_transform method of the OneHotEncoder class is called with the 'Country' column of the X_train DataFrame as input to fit the one-hot encoder to the 'Country' column and create a one-hot encoded representation of it. This encoded column is stored in a new DataFrame called X_train_encoded.

The transform method of the OneHotEncoder class is then called with the 'Country' column of the X_test DataFrame as input to create a one-hot encoded representation of it. This encoded column is stored in a new DataFrame called X_test_encoded.

After performing these steps, the 'Country' column in both X_train_encoded and X_test_encoded will be replaced with their one-hot encoded versions, which allows them to be used as input features in a machine learning algorithm. One-hot encoding is necessary for machine learning algorithms that cannot work with categorical data directly, and it enables the data to be more easily interpreted and analyzed.

In [ ]:
# Encode the 'Country' column using one-hot encoding.
from sklearn.preprocessing import OneHotEncoder

# Create the encoder
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[['Country']])
X_test_encoded = encoder.transform(X_test[['Country']])

In [ ]:
# Combine the encoded 'Country' column with the 'year' column
import numpy as np

# X_train_encoded.toarray() returns a numpy array with the one-hot encoded values
# X_train['year'].values.reshape(-1, 1) returns a numpy array with the year values
# np.hstack([X_train_encoded.toarray(), X_train['year'].values.reshape(-1, 1)]) returns a numpy array with the one-hot encoded values and the year values

X_train_final = np.hstack([X_train_encoded.toarray(), X_train['year'].values.reshape(-1, 1)])
X_test_final = np.hstack([X_test_encoded.toarray(), X_test['year'].values.reshape(-1, 1)])

In [ ]:
# Train a linear regression model

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_final, y_train)